In [1]:
import numpy as np
import pandas as pd
import re
import string
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [2]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [3]:
vocab = pd.read_csv('../static/model/imdb_vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [6]:
len(tokens)

533

In [8]:
import pickle
with open('../static/model/imdb_lr_model.pickle', 'rb') as f:
    model = pickle.load(f)

In [20]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '' ,x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [10]:
def vectorizer(df, vocabulary):
    vectorized_lst = []

    for sentence in df:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
        vectorized_lst.append(sentence_lst)
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new
    

## Pipeline

In [18]:
text  = "really enjoyed it"

In [12]:
def get_prediction(vect_text):
    prediction = model.predict(vect_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [19]:
preprocessed_text = preprocessing(text)
vectorized_text = vectorizer(preprocessed_text, tokens)
prediction = get_prediction(vectorized_text)
prediction

'positive'